In [15]:
import torch
import pandas as pd

from datasets import Dataset
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [16]:
#모델 및 토크나이저 로드
checkpoint = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
#데이터셋 로드
train_data = pd.read_csv("datas/ratings_train.txt", names=["id", "document", "label"], skiprows=1, sep="\t")
test_data = pd.read_csv("datas/ratings_test.txt", names=["id", "document", "label"], skiprows=1, sep="\t")

train_data = train_data.dropna(axis=0)
test_data = test_data.dropna(axis=0)

train_datasets = Dataset.from_pandas(train_data)
test_datasets = Dataset.from_pandas(test_data)
#raw_datasets = Dataset(train_datasets+test_datasets)

In [18]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [19]:
train_datasets[0]

{'id': 9976970,
 'document': '아 더빙.. 진짜 짜증나네요 목소리',
 'label': 0,
 '__index_level_0__': 0}

In [20]:
train_datasets.features

{'id': Value(dtype='int64', id=None),
 'document': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [21]:
tokenizer_sentences1 = tokenizer(train_datasets['document'])

In [22]:
inputs = tokenizer("아니 진짜 노잼이네")
inputs

{'input_ids': [9320, 23971, 9153, 8174, 8146, 7098], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [23]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['▁아니', '▁진짜', '▁노', '잼', '이', '네']